In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import librosa
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,LSTM,GlobalMaxPool1D, Bidirectional 
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn import metrics
import pickle
from scipy.io.wavfile import read,write
import pickle

from datetime import datetime
from packaging import version

tf.keras.backend.set_floatx('float64')

print("TensorFlow version: ", tf.__version__)

os.chdir('drive/My Drive/')

TensorFlow version:  2.2.0


In [0]:
print(tf.config.list_physical_devices('GPU'))

import warnings
warnings.filterwarnings('ignore')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [0]:
!pwd
os.listdir()
# os.chdir('My Drive')

/content/drive/My Drive


['trimmed_audio',
 'Help_Data_Natural_10dB',
 'Help_Data_Old',
 'Help_Data_10dB',
 'Help_Data_Test_Old',
 'Negative_Data_10dB',
 'Negative_Data',
 'Negative_Data_20dB',
 'Negative_Data_Test_Old',
 'Negative_Data_Natural_10dB',
 'Colab Notebooks',
 'help_trimmed_train_x.pickle',
 'help_trimmed_train_y.pickle',
 'help_trimmed_test_x.pickle',
 'help_trimmed_test_y.pickle',
 'help_trimmed.h5',
 'logs']

In [0]:
activ_dir = 'Help_Data_Old'
fs,x = read(activ_dir + '/' + '1_1.wav')
file_size = x.shape[0]
segment_time = 1.0
segment_samples = int(segment_time * fs)
no_of_segments = int(file_size/segment_samples)


# for i in range(no_of_segments - 1):
#   file_name = '{}_{}.wav'.format(activ_dir, i)
#   x_temp = x[i*segment_samples:(i+1)*segment_samples]
#   write(file_name, fs, x_temp)


print(fs)
print(x.shape)
print(file_size)
print(segment_samples)
print(no_of_segments)

16000
(16000,)
16000
16000
1


In [0]:
# KEYWORD_FOLDER1= 'Bachao_Data_Old/'
# KEYWORD_FOLDER2= 'Bachao_Data_Babble_10dB'
# KEYWORD_FOLDER3 = 'Bachao_Data_Natural_10dB'

KEYWORD_FOLDER4 = 'Help_Data_Old'
KEYWORD_FOLDER5 = 'Help_Data_10dB'
KEYWORD_FOLDER6= 'Help_Data_Natural_10dB'


NEGATIVE_FOLDER1 = 'Negative_Data/'
NEGATIVE_FOLDER2 = 'Negative_Data_10dB'
NEGATIVE_FOLDER3 = 'Negative_Data_Natural_10dB'

# NEGATIVE_FOLDER4 = 'Negative_Data/'
# NEGATIVE_FOLDER5 = 'Negative_Data_10dB'
# NEGATIVE_FOLDER6 = 'Negative_Data_Natural_10dB'

#OPPPOSITE_KEYWORD_FOLDER = 'Bachao_Data/'
KEYWORD_FOLDER_TEST = 'Help_Data_Test_Old/'
NEGATIVE_FOLDER_TEST = 'Negative_Data_Test_Old/'
#OPPPOSITE_KEYWORD_FOLDER_TEST = 'Bachao_Data_Test/'

In [0]:
INPUT_SHAPE = (126, 40)
dropout = 0.2
var = 0

In [0]:
def count_files(folder, extension):
	count = 0
	for file in os.listdir(folder):
		if file.endswith(extension):
			file_path = os.path.join(folder, file)
			count += 1
	return count

In [0]:
def load_data_folder(folder, is_keyword):
  num_samples = count_files(folder, '.wav')
  data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
  data_Y = np.zeros((num_samples), dtype=np.float64)

  count = 0
  for file in os.listdir(folder):
    if file.endswith('.wav'):
      file_path = os.path.join(folder, file)
      y, sr = librosa.load(file_path,sr=None)
      mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=128, n_fft=256, n_mfcc=20)
      mfcc_delta = librosa.feature.delta(mfcc)[:10, :]
      mfcc_double_delta = librosa.feature.delta(mfcc, order=2)[:10, :]
      data_X[count, :, :20] = mfcc.T
      data_X[count, :, 20:30] = mfcc_delta.T
      data_X[count, :, 30:] = mfcc_double_delta.T
      data_Y[count] = int(is_keyword)
      count += 1
      if count%50==0:
        print(count)
  return data_X, data_Y

In [0]:
def load_data(folders):
	num_samples = sum([count_files(folder, '.wav') for folder, is_keyword in folders])
	data_X = np.zeros((num_samples, INPUT_SHAPE[0], INPUT_SHAPE[1]), dtype=np.float64)
	data_Y = np.zeros((num_samples), dtype=np.float64)
	count = 0
	for folder, is_keyword in folders:
		num_samples_folder = count_files(folder, '.wav')
		data_X[count:count+num_samples_folder, :, :], data_Y[count:count+num_samples_folder] = (
			load_data_folder(folder, is_keyword))
		count += num_samples_folder
	return shuffle(data_X, data_Y, random_state=0)


In [0]:
def load_train_data():
  #folders = [(NEGATIVE_FOLDER_TRAIN_1, False), (NEGATIVE_FOLDER_TRAIN_2, False), (NEGATIVE_FOLDER_TRAIN_3, False)]
  # folders = [(KEYWORD_FOLDER4, True), (KEYWORD_FOLDER5, True), (NEGATIVE_FOLDER1, False), (NEGATIVE_FOLDER2, False), (NEGATIVE_FOLDER3, False), (NEGATIVE_FOLDER4, False)]
  folders = [(KEYWORD_FOLDER4, True), (KEYWORD_FOLDER5, True), (KEYWORD_FOLDER6, True), (NEGATIVE_FOLDER1, False), (NEGATIVE_FOLDER2, False), (NEGATIVE_FOLDER3, False)]
  return load_data(folders)

In [0]:
def load_test_data():
  #folders = [(NEGATIVE_FOLDER_TEST, True)]
  folders = [(KEYWORD_FOLDER_TEST, True), (NEGATIVE_FOLDER_TEST, False)] 
  return load_data(folders)

In [0]:
#Generate the MFCC matrix
train_X0, train_Y0 = load_train_data()
print("Train data extracted")

test_X, test_Y = load_test_data()
print("Test data extracted")


In [0]:
# Dump into a pickle

import pickle

with open("help_trimmed_train_x.pickle", "wb") as f:
  pickle.dump(train_X0, f)
print("Train set features done")
with open("help_trimmed_train_y.pickle", "wb") as f:
  pickle.dump(train_Y0, f)
print("Train set labels done")


with open("help_trimmed_test_x.pickle", "wb") as f:
  pickle.dump(test_X, f)
print("Test set features done")
with open("help_trimmed_test_y.pickle", "wb") as f:
  pickle.dump(test_Y, f)
print("Test set labels done")

Train set features done
Train set labels done
Test set features done
Test set labels done


In [0]:
# Load from a pickle

with open('help_trimmed_train_x.pickle', 'rb') as f:
  train_X0 = pickle.load(f)
print("Train set features loaded")
with open('help_trimmed_train_y.pickle', 'rb') as f:
  train_Y0 = pickle.load(f)
print("Train set labels loaded")

with open('help_trimmed_test_x.pickle', 'rb') as f:
  test_X = pickle.load(f)
print("Test set features loaded")
with open('help_trimmed_test_y.pickle', 'rb') as f:
  test_Y = pickle.load(f)
print("Test set labels loaded")

Train set features loaded
Train set labels loaded
Test set features loaded
Test set labels loaded


In [0]:
train_X0.shape

(3606, 126, 40)

In [0]:
randomize = np.arange(len(train_Y0))
np.random.shuffle(randomize)
train_X_shuffle = train_X0[randomize]
train_Y_shuffle = train_Y0[randomize]

In [0]:
# print(test_X.shape)
# print(test_Y.shape)
print(train_X_shuffle.shape)
print(train_Y_shuffle.shape)

(3606, 126, 40)
(3606,)


In [0]:
import sklearn
train_X_cv, test_X_cv, train_Y_cv, test_Y_cv = sklearn.model_selection.train_test_split(train_X_shuffle, train_Y_shuffle, test_size=0.2)

In [0]:
train_X_cv.shape

(2884, 126, 40)

In [0]:
def create_model():
  print ('Creating model...')
  model = Sequential()
  model.add(Bidirectional(LSTM(units=128, input_shape=INPUT_SHAPE, return_sequences=True)))
  model.add(GlobalMaxPool1D())
  model.add(Dense(units=1,activation='sigmoid'))
  opt = keras.optimizers.Adam(learning_rate=0.001)	
  model.compile(loss='binary_crossentropy',
	              optimizer=opt,
	              metrics=['accuracy'])
  return model

In [0]:
model.summary()

NameError: ignored

In [0]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = create_model()
# model.fit(train_X_cv, train_Y_cv, epochs=5, validation_split=0.2)

training_history = model.fit(
    train_X_cv,
    train_Y_cv,
    verbose=1,
    epochs=9,
    validation_split=0.2,
    callbacks=[tensorboard_callback]
)

print("Average test loss: ", np.average(training_history.history['loss']))

Creating model...
Epoch 1/9
73/73 [==============================] - 4s 59ms/step - loss: 0.4593 - accuracy: 0.8010 - val_loss: 0.3403 - val_accuracy: 0.8666
Epoch 2/9
73/73 [==============================] - 3s 47ms/step - loss: 0.2992 - accuracy: 0.8912 - val_loss: 0.2346 - val_accuracy: 0.9116
Epoch 3/9
73/73 [==============================] - 3s 47ms/step - loss: 0.2004 - accuracy: 0.9341 - val_loss: 0.1674 - val_accuracy: 0.9393
Epoch 4/9
73/73 [==============================] - 3s 47ms/step - loss: 0.1440 - accuracy: 0.9645 - val_loss: 0.1310 - val_accuracy: 0.9549
Epoch 5/9
73/73 [==============================] - 3s 47ms/step - loss: 0.1131 - accuracy: 0.9684 - val_loss: 0.1097 - val_accuracy: 0.9584
Epoch 6/9
73/73 [==============================] - 3s 47ms/step - loss: 0.0795 - accuracy: 0.9848 - val_loss: 0.0844 - val_accuracy: 0.9688
Epoch 7/9
73/73 [==============================] - 3s 47ms/step - loss: 0.0611 - accuracy: 0.9896 - val_loss: 0.0672 - val_accuracy: 0.9775
Ep

In [0]:
%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir logs/scalars

In [0]:
predictions = model.predict(test_X)

In [0]:
y_pred = (predictions>0.6)

In [0]:
matrix = metrics.confusion_matrix(test_Y, y_pred)

In [0]:
matrix

array([[35,  4],
       [ 7, 93]])

In [0]:
#Uncomment to save the retrained model
model.save('help_trimmed.h5')

In [0]:
model = keras.models.load_model('help_model_10dB.h5')